In [9]:
!pip install pulp

In [10]:
import pandas as pd
from pulp import *
import numpy as np

In [11]:
df = pd.read_csv("product_sales.csv")

In [12]:
df.head()

,item_id,unit_price,unit_cost,units_sold,incr_cvr,incr_sales
0,1,95,92,0.6,0.323,0.08906
1,2,37,27,0.8,0.106,0.01991
2,3,34,26,0.6,0.328,0.03778
3,4,32,30,0.7,0.000,0.08562
4,5,62,45,1.0,0.079,0.02637


In [13]:
df['profit'] = df['units_sold']*(df['unit_price']-df['unit_cost'])*(1-df['incr_cvr'])*(1-df['incr_sales'])*100
df.head()

,item_id,unit_price,unit_cost,units_sold,incr_cvr,incr_sales,profit
0,1,95,92,0.6,0.323,0.08906,111.007148
1,2,37,27,0.8,0.106,0.01991,700.960368
2,3,34,26,0.6,0.328,0.03778,310.373683
3,4,32,30,0.7,0.000,0.08562,128.013200
4,5,62,45,1.0,0.079,0.02637,1524.412491


In [14]:
df[df['item_id']==100]

,item_id,unit_price,unit_cost,units_sold,incr_cvr,incr_sales,profit
99,100,66,53,2.0,0.401,0.04617,1485.494842


In [15]:
np.sum(df['units_sold']*(df['unit_price']-df['unit_cost']))

3285.9

In [20]:
products = list(df['item_id'])

volumes = dict(zip(products,df['units_sold']))
price = dict(zip(products,df['unit_price']))
cost = dict(zip(products,df['unit_cost']))
incr_cvr = dict(zip(products,df['incr_cvr']))
incr_sales = dict(zip(products,df['incr_sales']))

df['price_change_val'] = 0.199
price_change_val = df['price_change_val']
price_change_multiplier = 1 + price_change_val
adjusted_price = df['unit_price'] * price_change_multiplier
setAdjustedVolume(price_change_val, df['incr_sales'], df['incr_cvr'], df['units_sold'])
adjusted_volume = df['adjusted_volume']

sales_vars = LpVariable.dicts("Sales",products,0,cat='Continuous')

In [19]:
def setAdjustedVolume(price_change_val, incr_sales, incr_cvr, original_units_sold):
    df['perc_incr_units_sold'] = [incr_sales*abs(x) + incr_cvr*abs(x) if x < 0 else 0.05*abs(x) for x in df['price_change_val']]    
    df['adjusted_volume'] = original_units_sold*(1+df['perc_incr_units_sold'])
    

In [24]:
min_price = -25
max_price = 50

In [25]:
prob = LpProblem("PricetOpt",LpMaximize)

adjusted_profit=adjusted_price*adjusted_volume-df['unit_cost']*adjusted_volume

profit = dict(zip(products,adjusted_profit))
price_update = dict(zip(products,price_change_val))

prob += lpSum([profit[i]*sales_vars[i] for i in products]), "Total Profit"

prob += lpSum([price_update[f]*sales_vars[f]  for f in products]) >= min_price, "PriceChangeMin"
prob += lpSum([price_update[f]*sales_vars[f]  for f in products]) <= max_price, "PriceChangeMax"

#prob.writeLP("PricetOpt.lp")
prob.solve()
print("Status:", LpStatus[prob.status])



Status: Optimal


In [26]:
print("The total profit is: ${}".format(round(value(prob.objective),2)))

The total profit is: $33227.33


In [28]:
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Sales_204 = 251.25628


In [27]:
prob

PricetOpt:
MAXIMIZE
13.273772849999993*Sales_1 + 35.845145400000035*Sales_10 + 52.78806659999998*Sales_100 + 1.6314732300000006*Sales_101 + 31.313600744999995*Sales_102 + 5.431511100000002*Sales_103 + 28.021971705000013*Sales_104 + 11.226806190000005*Sales_105 + 37.38976293*Sales_106 + 117.90156300000001*Sales_107 + 59.68562112000001*Sales_108 + 7.597045890000004*Sales_109 + 15.583124520000005*Sales_11 + 31.92411552*Sales_110 + 45.8537499*Sales_111 + 34.25911991999999*Sales_112 + 43.84011159000002*Sales_113 + 11.380621575*Sales_114 + 40.18035577500001*Sales_115 + 27.365605200000005*Sales_116 + 1.8118502999999997*Sales_117 + 46.34660550000001*Sales_118 + 9.528575265000004*Sales_119 + 81.48781575000004*Sales_12 + 9.265281300000002*Sales_120 + 27.173209725000007*Sales_121 + 6.484686960000001*Sales_122 + 81.27905908500001*Sales_123 + 71.76260322000002*Sales_124 + 9.010975890000005*Sales_125 + 43.27352964000001*Sales_126 + 10.439045190000002*Sales_127 + 8.25028155*Sales_128 + 11.05390275000